In [1]:
## add ignore warnings for now, will remove and debug once full algorithm is complete
import warnings
warnings.filterwarnings("ignore")

## import packages/libraries
from time import perf_counter, clock_gettime_ns, CLOCK_REALTIME
import numpy as np
from multiprocessing import Pool, cpu_count
from itertools import product
import sys
import sqlite3

## append filepath to allow files to be called from within project folder
sys.path.append('/home/gerard/Desktop/capstone_project/patoms')
sys.path.append('/home/gerard/Desktop/capstone_project')

## call locally created functions
from snapshot_2d_pattern_v5 import patoms2d
from snapshot_3d_pattern_v6 import patoms3d
from pattern_2d_compare_v4 import pattern_compare_2d
from pattern_3d_compare_v4 import pattern_compare_3d
from get_2d_table_names_v0 import table_names_2d
from updating_ref_table_v0 import update_ref_table
from patom_to_table_v0 import patom_to_table_func

In [2]:
## create in memory 2d database
con2d = sqlite3.connect(":memory:")
cur2d = con2d.cursor()
cur2d.execute('create table pat_2d_000000(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time);')

In [3]:
## read 2d ref database into memory and convert to ref patoms for comparison
working_ref_patoms = []
with open('/home/gerard/Desktop/capstone_project/ref2d.db', 'rb') as f:
    con2dref = sqlite3.connect(":memory:")
    con2dref.deserialize(f.read())
    cur2dref = con2dref.cursor()
    ref_names = [name for (name,) in cur2dref.execute("select name from sqlite_master where type='table';").fetchall()]
    for i in ref_names:
        table = cur2dref.execute(f"select * from {i};").fetchall()
        # patom = [norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, patom_ind, frame_ind, patom_time]
        norm_x = [x[0] for x in table]; norm_y = [x[1] for x in table]; pattern_centroid_x = [x[2] for x in table][0]; pattern_centroid_y = [x[3] for x in table][0]
        working_ref_patoms.append([norm_x, norm_y, pattern_centroid_x, pattern_centroid_y])

In [4]:
val = 0
while val == 0:
    rand_array = np.random.random((1, 720, 1280))
    y_len = rand_array.shape[1]
    x_len = rand_array.shape[2]
    # ingest data frame by frame
    for frame in range(rand_array.shape[0]):
        #################################################################################
        ####################### FIRST TASK: FIND PATTERNS IN FRAME ######################
        #################################################################################
        frame_patoms = patoms2d(x_len, y_len, rand_array[frame,:,:], frame)
        # patom = [[norm_x, norm_y], [pattern_centroid_x, pattern_centroid_y], patom_ind, frame_ind, patom_time]
        # patom[i][[0][0]: list of x_pos, patom[i][[0][1]: list of y_pos, patom[i][[1][0]: x_cent, patom[i][[1][1]: y_cent, patom[i][[2]: patom_ind, patom[i][[3]: frame_ind, patom[i][[4]: patom_time,
        num_patoms = len(frame_patoms)
        val += 1
        ###########################################################################
        ########## SECOND TASK: COMPARE NEW PATOMS AGAINST REF PATOMS #############
        ###########################################################################
        atime = perf_counter()
        comp_indexes = list(product(range(num_patoms), range(len(working_ref_patoms[:100]))))
        with Pool(processes=cpu_count()) as pool:
            items = [(frame_patoms[i[0]][0], frame_patoms[i[0]][1], frame_patoms[i[0]][2], frame_patoms[i[0]][3],\
                    working_ref_patoms[i[1]][0], working_ref_patoms[i[1]][1], working_ref_patoms[i[1]][2], working_ref_patoms[i[1]][3], i) for i in comp_indexes]
            ## function outputs ind value of the patom_indexes list, the centroid and distance similarity measures
            comp_results = pool.starmap(pattern_compare_2d, items)
            comp_results = [x for x in comp_results if x is not None]
            print("Time to compare 2D patterns with multiprocessing (secs):", (perf_counter()-atime))
            # res output: [(i, j)]
            ## loop through the output of the comparison function
            if comp_results:
                for ix, i in enumerate(comp_results):
                    print('similar')
                    # get max table number from 2d database
                    mttime = perf_counter()
                    next_table_num = int([table for (table,) in cur2d.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()].pop(-1)[-6:]) + 1 
                    print('time to get max table:',perf_counter()-mttime)
                    next_table_num = str(next_table_num).zfill(6)
                    # check if table exists, if so, add patom, if not create table and add patom
                    ref_table_num = ref_names[i[1]][-6:] # check this is correct
                    new_patom = frame_patoms[i[0]] # check this is correct
                    ptttime = perf_counter()
                    patom_to_table = patom_to_table_func(new_patom)
                    print('time to convert patom to table:',perf_counter()-ptttime)
                    # add patom to relevant pattern table get name of ref table to get data table name
                    if f"pat_2d_{ref_table_num}" in [names for (names,) in cur2d.execute("select name from sqlite_master where type='table';").fetchall()]:
                        cur2d.executemany(f"INSERT INTO pat_2d_{ref_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
                        con2d.commit()
                        # patom = [norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, patom_ind, frame_ind, patom_time]
                    else:
                        cur2d.execute(f"CREATE TABLE pat_2d_{ref_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time);")
                        cur2d.executemany(f"INSERT INTO pat_2d_{ref_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
                        con2d.commit()
            else:
                for i in frame_patoms:
                    mttime = perf_counter()
                    next_table_num = int([table for (table,) in cur2d.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()].pop(-1)[-6:]) + 1 
                    print('time to get max table:',perf_counter()-mttime)
                    next_table_num = str(next_table_num).zfill(6)
                    new_patom = i # check this is correct
                    ptttime = perf_counter()
                    patom_to_table = patom_to_table_func(new_patom)
                    print('time to convert patom to table:',perf_counter()-ptttime)
                    # if patom is not matched then create new pattern & ref tables, get max table name and then create next new one
                    cur2d.execute(f"CREATE TABLE pat_2d_{next_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time);")
                    cur2d.executemany(f"INSERT INTO pat_2d_{next_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
                    con2d.commit()


Time to get 2D patterns with multiprocessing (secs): 0.434321534994524
Time to compare 2D patterns with multiprocessing (secs): 1.718040321022272
similar
time to get max table: 0.0019492849824018776
time to convert patom to table: 0.00021464802557602525
similar
time to get max table: 8.453300688415766e-05
time to convert patom to table: 0.00010257295798510313
similar
time to get max table: 9.399495320394635e-05
time to convert patom to table: 7.107702549546957e-05
similar
time to get max table: 4.386401269584894e-05
time to convert patom to table: 4.853698192164302e-05
similar
time to get max table: 3.830395871773362e-05
time to convert patom to table: 4.2896950617432594e-05
similar
time to get max table: 3.7797028198838234e-05
time to convert patom to table: 3.877899143844843e-05
similar
time to get max table: 3.8646976463496685e-05
time to convert patom to table: 2.154201501980424e-05
similar
time to get max table: 1.7012993339449167e-05
time to convert patom to table: 9.848969057202

In [28]:
table = [table for (table,) in cur2d.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()][0]
print(table)
pat_table = cur2d.execute(f"select * from {table};").fetchall()
ref_table = cur2dref.execute(f"select * from pat_2d_ref_{table[-6:]};").fetchall()
# for ix, i in enumerate(pat_table):
#     for jx, j in enumerate(ref_table):
#         print(pat_table[ix][0], ref_table[jx][0])
print(len(pat_table))
print(len(ref_table))

pat_2d_000000
59
153


In [6]:
# patom_table_names = cur2d.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE '%ref%';").fetchall()
# patom_tables = [name for (name,) in patom_table_names]
# pat_cnts = []
# for i in patom_tables:
#     frame_cnt = cur2d.execute(f"select count(distinct patom_time) as cnt_ from {i};").fetchone()
#     pat_cnts.append(frame_cnt)

# print(max(pat_cnts))

# con2d.close()